# LangChain Introduction

LangChain 是圍繞 LLMs 構建的框架。我們可以將其用於聊天機器人、Question-Answering (QA)、摘要等等。

這個函式庫的核心思想是我們可以將不同的元件 “鏈結” 在一起，以創建更多元的 LLMs 應用。 Chain 來自幾個 Module 的多個組件：

1. **Prompt templates**：Prompt templates 是不同類型提示的範本。例如「 chatbot 」樣式模板、ELI5 問答等
2. **LLMs**：像 GPT-3、Mistral、Llama、Breeze、TAIDE 等大型語言模型
3. **Agents**：Agents 使用 LLMs 決定應採取的動作。可以使用網路搜尋(Google Search)或計算器(Python func)之類的工具，並將所有工具包裝成一個邏輯循環的操作。
4. **Memory**：短期記憶、長期記憶。

我們將從 Prompt templates 和 LLMs 的基礎知識開始。以下教學將提供兩個 LLMs 選項，包含 Hugging Face Hub 或 Hugging Face Pipeline 的模型。

- GitHub: https://github.com/langchain-ai/langchain
- Docs: https://python.langchain.com/en/latest/index.html

In [ ]:
import torch
from transformers import BitsAndBytesConfig

from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain.schema import StrOutputParser
from langchain_huggingface import ChatHuggingFace

In [ ]:
from huggingface_hub import notebook_login

# chat model 需要使用 hf 的 token
notebook_login()

### 1. LLMs
透過 Langchain 載入 Huggingface 上的各種大型語言模型，在 Langchain 內模型可以分為

1. LLM 模式：給予文字輸入，然後文字輸出

2. Chat Models 模式：基於LLM模式的更進階的模式，他的輸入和輸出是格式化的chat messages

In [ ]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

# chat model 初始化
chat_llm = ChatHuggingFace(llm=llm)

In [ ]:
prompt = """
提問: 2022 年全球最賣座的電影是哪一部?

Let's think step by step.

解答: """
print(llm(prompt))

In [ ]:
prompt = """
提問: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

解答: """
print(llm(prompt))

### 2. Prompt
一個好的 Prompt 通常包含以下四個組成部分：

1. **指示**: 告訴模型要做什麼，如何使用提供的信息，如何處理查詢，並建立輸出
2. **範例輸入**: 提供範例輸入，以向模型示範預期的內容
3. **範例輸出**: 提供對應的範例輸出
4. **查詢**: 您希望模型處理的實際輸入

以下介紹幾種在 LangChain 上使用 Pormpt 的方式

### 2.1 PromptTemplate

In [ ]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我一個笑話")
message = prompt_template.format()

# 模型生成
print(llm(message))

In [ ]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我關於一個{content}的{adjective}笑話。")
message = prompt_template.format(adjective="悲傷的", content="數據科學家")

# 模型生成
print(llm(message))

### 2.2 ChatPromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個擁有強大能力的 AI 機器人。你的名字是{name}。"),
        ("human", "你好，你好嗎？"),
        ("ai", "我很好，謝謝！"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(name="Bob", user_input="你叫什麼名字？")
print(chat_llm.invoke(messages).content)

### 2.3 混合使用

In [ ]:
system_prompt = "你是一個擁有強大能力的 AI 機器人。"
question_prompt = "告訴我關於一個{content}的{adjective}笑話。"

full_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_prompt),
    HumanMessage("你好，你好嗎？"),
    AIMessage("我很好，謝謝！"),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=question_prompt,
            input_variables=["content", "adjective"])
    )
])
full_prompt.pretty_print()

## 3. LLM Chain
對於簡單的任務，使用單一 LLM（大型語言模型）效果很好。然而，對於更複雜的任務，通常需要鍊式多個步驟和/或模型。

在LangChain中，可以使用傳統的 LLMChain，較新且建議的方法是 LangChain 表達式語言（LCEL）。

### 3.1 LLMChain

In [ ]:
template = """問題: {question}

Let's think step by step.

答案: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 將問題透過參數化的方式帶入
question = "NBA 2023 年總冠軍球隊是誰?"
print(llm_chain.invoke({"question": question})["text"])

### 3.2 LCEL（LangChain Expression Language）

In [ ]:
# 需要加入 Instruction 參數，才能告訴模型你的對話結束了，因目前使用的模型上不支援 chat 模式
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位專業的資料科學家和機器學習工程師，能夠提供專業的知識並準確地回答問題。",),
        ("human", "{question}"),
    ]
)

# 使用 LCEL 將 Prompt 與 LLM 等串接起來
llm_chain = prompt | chat_llm | StrOutputParser()

# 將問題透過參數化的方式帶入
print(llm_chain.invoke({"question": "機器學習和深度學習有什麼不同？"}))